<a href="https://colab.research.google.com/github/MichaelGelo/GRP2_DeepDive__CUDA/blob/main/DeepDive_CUDA_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group 2 - DeepDive CUDA - Histogram counting (Atomic operations)**
## **GROUP 2 - S11**

**MEMBERS:**

- Alfred Bastin S. Agustines
- Allan David C. De Leon
- Michael Angelo Depasucat
- Kai Hiori J. Padilla


# Check if CUDA is present

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb 19 18:55:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# (1) C Histogram program (RAW)

In [ ]:
%%writefile C_histo.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <stdint.h>

#define NUM_BINS 10

void getHistogram(int* vector, int vectorSize, int* histogram) {
    for (int i = 0; i < vectorSize; i++) {
        int bIndex = vector[i] % 10;
        histogram[bIndex]++;
    }
}

int main() {
    const size_t ARRAY_SIZE = 1<<28;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);

    int32_t *vector;
    vector = (int32_t*)malloc(ARRAY_BYTES);
    int histogram[NUM_BINS] = { 0 };
    const size_t loope = 30;

    for (int i = 0; i < ARRAY_SIZE; i++) {
        vector[i] = i;
    }

    clock_t start, end;
    getHistogram(vector, ARRAY_SIZE, histogram);


    for (int i = 0; i < 10; i++) {
        histogram[i] = 0;
    }

    double elapse, time_taken;
    elapse = 0.0f;
    for (int i = 0; i < loope; i++) {
        for (int i = 0; i < 10; i++) {
            histogram[i] = 0;
        }
        start = clock();
        getHistogram(vector, ARRAY_SIZE, histogram);
        end = clock();
        time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
    }
    printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse / loope, ARRAY_SIZE);

    size_t err_count = 0;
    int32_t  test[NUM_BINS] = { 0 };

    for (int i = 0; i < ARRAY_SIZE; i++) {
        int bIndex = vector[i] % 10;
        test[bIndex]++;
    }
    for (int i = 0; i < 10; i++) {

        if (test[i] != histogram[i])
            err_count++;
    }
    printf("Error count (C program): %lu\n", err_count);

    for (int i = 0; i < NUM_BINS; i++) {
        printf("Bin %d: %d\n", i, histogram[i]);
    }

    free(vector);
	return 0;
}

Overwriting C_histo.c


In [ ]:
%%shell
gcc C_histo.c -o C_histo

In [ ]:
%%shell
./C_histo

Function (in C) average time for 30 loops is 1172.042167 milliseconds to execute an array size 268435456 
Error count (C program): 0
Bin 0: 26843546
Bin 1: 26843546
Bin 2: 26843546
Bin 3: 26843546
Bin 4: 26843546
Bin 5: 26843546
Bin 6: 26843545
Bin 7: 26843545
Bin 8: 26843545
Bin 9: 26843545


# (2) C Histogram program (ATOMIC)

In [1]:
%%writefile C_histo_atomic.c
#include <stdio.h>
#include <stdlib.h>
#include <stdatomic.h>
#include <time.h>
#include <stdint.h>

#define NUM_BINS 10

void getHistogram(int* vector, int vectorSize, atomic_int* histogram) {
    for (int i = 0; i < vectorSize; i++) {
        int bIndex = vector[i] % NUM_BINS;
        atomic_fetch_add(&histogram[bIndex], 1);
    }
}

int main() {
    const size_t ARRAY_SIZE = 1<<28;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);

    int32_t *vector;
    vector = (int32_t*)malloc(ARRAY_BYTES);
    atomic_int histogram[NUM_BINS] = { 0 };
    const size_t loope = 30;

    for (int i = 0; i < ARRAY_SIZE; i++) {
        vector[i] = i;
    }

    clock_t start, end;
    getHistogram(vector, ARRAY_SIZE, histogram);


    for (int i = 0; i < 10; i++) {
        histogram[i] = 0;
    }

    double elapse, time_taken;
    elapse = 0.0f;
    for (int i = 0; i < loope; i++) {
        for (int i = 0; i < 10; i++) {
            histogram[i] = 0;
        }
        start = clock();
        getHistogram(vector, ARRAY_SIZE, histogram);
        end = clock();
        time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
    }
    printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse / loope, ARRAY_SIZE);

    size_t err_count = 0;
    int32_t  test[NUM_BINS] = { 0 };

    for (int i = 0; i < ARRAY_SIZE; i++) {
        int bIndex = vector[i] % 10;
        test[bIndex]++;
    }
    for (int i = 0; i < 10; i++) {

        if (test[i] != histogram[i])
            err_count++;
    }
    printf("Error count (C program): %lu\n", err_count);

    for (int i = 0; i < NUM_BINS; i++) {
        printf("Bin %d: %d\n", i, histogram[i]);
    }

    free(vector);
	return 0;
}

Writing C_histo_atomic.c


In [2]:
%%shell
gcc C_histo_atomic.c -o C_histo_atomic

In [3]:
%%shell
./C_histo_atomic

Function (in C) average time for 30 loops is 2333.543200 milliseconds to execute an array size 268435456 
Error count (C program): 0
Bin 0: 26843546
Bin 1: 26843546
Bin 2: 26843546
Bin 3: 26843546
Bin 4: 26843546
Bin 5: 26843546
Bin 6: 26843545
Bin 7: 26843545
Bin 8: 26843545
Bin 9: 26843545


# (3) CUDA program w/ Unified memory, pre-fetching and memadvise (RAW)

In [10]:
%%writefile cuda_raw.cu
#include <stdio.h>
#include <stdlib.h>

#define VECTOR_SIZE (1 << 28)
#define NUM_LOOPE 30
#define NUM_BINS 10

// CUDA kernel using atomic operations
__global__ void histogramKernel(int *d_data, int *d_histogram, int size) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = index; i < size; i += stride) {
        int bin = d_data[i] % NUM_BINS;
        d_histogram[bin]++;
    }
}

int main() {
    const size_t ARRAY_SIZE = VECTOR_SIZE;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(int);

    //number of times the program is to be executed
    const size_t loope = NUM_LOOPE;

    //declare array
    int *data, *histogram;
    cudaMallocManaged(&data, ARRAY_BYTES);
    cudaMallocManaged(&histogram, NUM_BINS * sizeof(int));

    //get gpu id
    int device = -1;
    cudaGetDevice(&device);

    // memory advise
    cudaMemAdvise(data, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(data, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

    //"prefetch data" to create CPU page memory
    cudaMemPrefetchAsync(data, ARRAY_BYTES, cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(histogram, NUM_BINS * sizeof(int), device, NULL);

    // *** init array
    for (size_t i = 0; i < VECTOR_SIZE; i++) {
        data[i] = i;
    }

    //"Prefetch data" from CPU-GPU
    cudaMemPrefetchAsync(data, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(histogram, NUM_BINS * sizeof(int), device, NULL);

    // *** setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (VECTOR_SIZE + numThreads - 1) / numThreads;

    printf("*** function = Histogram Counting\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu \n", numBlocks, numThreads);

    for (size_t i = 0; i < loope; i++) {
        cudaMemset(histogram, 0, NUM_BINS * sizeof(int));
        histogramKernel<<<numBlocks, numThreads>>>(data, histogram, VECTOR_SIZE);
    }

    //barrier
    cudaDeviceSynchronize();

    //"Prefetch data" from GPU-CPU
    cudaMemPrefetchAsync(histogram, NUM_BINS * sizeof(int), cudaCpuDeviceId, NULL);

    //error checking
    size_t err_count = 0;
    int referenceHistogram[NUM_BINS] = {0};
    for (size_t i = 0; i < VECTOR_SIZE; i++) {
        int bin = data[i] % NUM_BINS;
        referenceHistogram[bin]++;
    }
    for (int i = 0; i < NUM_BINS; i++) {
        if (histogram[i] != referenceHistogram[i]) {
            printf("Mismatch at bin %d: CUDA=%d, CPU=%d\n", i, histogram[i], referenceHistogram[i]);
            err_count++;
        }
    }
    printf("Error count (CUDA program): %zu\n", err_count);

    // Print results
    for (int i = 0; i < NUM_BINS; i++) {
        printf("Bin %d: %d\n", i, histogram[i]);
    }

    //free memory
    cudaFree(data);
    cudaFree(histogram);

    return 0;
}

Overwriting cuda_raw.cu


In [11]:
%%shell
nvcc -o cuda_raw cuda_raw.cu -arch=sm_75

In [12]:
%%shell
nvprof ./cuda_raw

==4256== NVPROF is profiling process 4256, command: ./cuda_raw
*** function = Histogram Counting
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Mismatch at bin 0: CUDA=27427, CPU=26843546
Mismatch at bin 1: CUDA=27427, CPU=26843546
Mismatch at bin 2: CUDA=27427, CPU=26843546
Mismatch at bin 3: CUDA=27427, CPU=26843546
Mismatch at bin 4: CUDA=27427, CPU=26843546
Mismatch at bin 5: CUDA=27427, CPU=26843546
Mismatch at bin 6: CUDA=27427, CPU=26843545
Mismatch at bin 7: CUDA=27427, CPU=26843545
Mismatch at bin 8: CUDA=27425, CPU=26843545
Mismatch at bin 9: CUDA=27425, CPU=26843545
Error count (CUDA program): 10
Bin 0: 27427
Bin 1: 27427
Bin 2: 27427
Bin 3: 27427
Bin 4: 27427
Bin 5: 27427
Bin 6: 27427
Bin 7: 27427
Bin 8: 27425
Bin 9: 27425
==4256== Profiling application: ./cuda_raw
==4256== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.98%  314.75ms        30  10.492ms  7.4570ms  14.300ms  histogramK

# (4) CUDA program w/ Unified memory, pre-fetching and memadvise (Atomic Operations)

In [13]:
%%writefile cuda.cu
#include <stdio.h>
#include <stdlib.h>

#define VECTOR_SIZE (1 << 28)
#define NUM_LOOPE 30
#define NUM_BINS 10

// CUDA kernel using atomic operations
__global__ void histogramKernel(int *d_data, int *d_histogram, int size) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = index; i < size; i += stride) {
        int bin = d_data[i] % NUM_BINS;
        atomicAdd(&d_histogram[bin], 1);
    }
}

int main() {
    const size_t ARRAY_SIZE = VECTOR_SIZE;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(int);

    //number of times the program is to be executed
    const size_t loope = NUM_LOOPE;

    //declare array
    int *data, *histogram;
    cudaMallocManaged(&data, ARRAY_BYTES);
    cudaMallocManaged(&histogram, NUM_BINS * sizeof(int));

    //get gpu id
    int device = -1;
    cudaGetDevice(&device);

    // memory advise
    cudaMemAdvise(data, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(data, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

    //"prefetch data" to create CPU page memory
    cudaMemPrefetchAsync(data, ARRAY_BYTES, cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(histogram, NUM_BINS * sizeof(int), device, NULL);

    // *** init array
    for (size_t i = 0; i < VECTOR_SIZE; i++) {
        data[i] = i;
    }

    //"Prefetch data" from CPU-GPU
    cudaMemPrefetchAsync(data, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(histogram, NUM_BINS * sizeof(int), device, NULL);

    // *** setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (VECTOR_SIZE + numThreads - 1) / numThreads;

    printf("*** function = Histogram Counting\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu \n", numBlocks, numThreads);

    for (size_t i = 0; i < loope; i++) {
        cudaMemset(histogram, 0, NUM_BINS * sizeof(int));
        histogramKernel<<<numBlocks, numThreads>>>(data, histogram, VECTOR_SIZE);
    }

    //barrier
    cudaDeviceSynchronize();

    //"Prefetch data" from GPU-CPU
    cudaMemPrefetchAsync(histogram, NUM_BINS * sizeof(int), cudaCpuDeviceId, NULL);

    //error checking
    size_t err_count = 0;
    int referenceHistogram[NUM_BINS] = {0};
    for (size_t i = 0; i < VECTOR_SIZE; i++) {
        int bin = data[i] % NUM_BINS;
        referenceHistogram[bin]++;
    }
    for (int i = 0; i < NUM_BINS; i++) {
        if (histogram[i] != referenceHistogram[i]) {
            printf("Mismatch at bin %d: CUDA=%d, CPU=%d\n", i, histogram[i], referenceHistogram[i]);
            err_count++;
        }
    }
    printf("Error count (CUDA program): %zu\n", err_count);

    // Print results
    for (int i = 0; i < NUM_BINS; i++) {
        printf("Bin %d: %d\n", i, histogram[i]);
    }

    //free memory
    cudaFree(data);
    cudaFree(histogram);

    return 0;
}

Writing cuda.cu


In [14]:
%%shell
nvcc -o cuda cuda.cu -arch=sm_75

In [15]:
%%shell
nvprof ./cuda

==4500== NVPROF is profiling process 4500, command: ./cuda
*** function = Histogram Counting
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Error count (CUDA program): 0
Bin 0: 26843546
Bin 1: 26843546
Bin 2: 26843546
Bin 3: 26843546
Bin 4: 26843546
Bin 5: 26843546
Bin 6: 26843545
Bin 7: 26843545
Bin 8: 26843545
Bin 9: 26843545
==4500== Profiling application: ./cuda
==4500== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  1.92632s        30  64.211ms  61.074ms  108.18ms  histogramKernel(int*, int*, int)
                    0.00%  52.959us        30  1.7650us  1.5680us  4.0640us  [CUDA memset]
      API calls:   73.76%  1.86571s        30  62.190ms  84.362us  108.20ms  cudaMemset
                   13.43%  339.82ms         5  67.964ms  28.404us  235.95ms  cudaMemPrefetchAsync
                    8.65%  218.83ms         2  109.42ms  93.372us  218.74ms  cudaMallocManaged
                    2.41